In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import yellowbrick as yb
from matplotlib.colors import ListedColormap
from yellowbrick.classifier import ROCAUC
from matplotlib_venn import venn3
import matplotlib.patches as mpatches
from scipy.stats import normaltest, skew
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error, accuracy_score, f1_score
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.metrics import classification_report, confusion_matrix
from scipy.special import boxcox, inv_boxcox
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, cross_val_predict,  KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import PCA
from scipy.stats import zscore
from itertools import combinations
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import IsolationForest
import kmapper as km
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")
color= "Spectral"
color_plt = ListedColormap(sns.color_palette(color).as_hex())
color_hist = 'teal'
two_colors = [ sns.color_palette(color)[0], sns.color_palette(color)[5]]
three_colors = [ sns.color_palette(color)[5],sns.color_palette(color)[2], sns.color_palette(color)[0]]

In [ ]:
df = pd.read_csv('/kaggle/input/breast-cancer-gene-expression-profiles-metabric/METABRIC_RNA_Mutation.csv')

In [ ]:
# Only clinical attributes/variables mentioned in the problem description will be used 
df2= df.loc[:,"patient_id":"death_from_cancer"]

# shape of data
df2.shape

In [ ]:
# take a snip of data
df2.head()

# Data Cleaning 

In this section, data cleaning will be done to check and clean the data if needed so the data is ready to use

In [ ]:
# checking percentage of missing value
print(df2.isnull().sum()/len(df2)*100)

Some variables have missing values while others not.

# Exploratory Data Analysis 

In this section the variables will be explored to get insights from data

In [ ]:
# Numeric variable
# function that takes a dataframe and transforms it into a standard form after dropping nun_numirical columns
def to_standard (df2):
    
    num_df = df2[df2.select_dtypes(include = np.number).columns.tolist()]
    
    ss = StandardScaler()
    std = ss.fit_transform(num_df)
    
    std_df = pd.DataFrame(std, index = num_df.index, columns = num_df.columns)
    return std_df

In [ ]:
#Plot the boxplots of numerical variables
ax, fig = plt.subplots(1, 1, figsize = (20, 8))
plt.title('The Distribution of Numerical Attributes', fontsize = 18) 

sns.boxplot(y = "variable", x = "value", data = pd.melt(to_standard(df2)), palette = 'Spectral')
plt.xlabel('Range after Standarization', size = 16)
plt.ylabel('Variables', size = 16)

By looking at the boxplots, some variables have outlier(s) such as chemotherapy, lymph_nodes_examined_positive, mutation_count, tumor_size, tumor_stage

The distribution of the two target classes in numerical columns in the dataframe

In [ ]:
fig = plt.figure(figsize = (20, 30))
j = 0
num_cols = ['age_at_diagnosis', 'lymph_nodes_examined_positive','mutation_count', 'nottingham_prognostic_index', 'overall_survival_months', 'tumor_size' ]
for i in df[num_cols].columns:
    plt.subplot(6, 4, j+1)
    j += 1
    sns.distplot(df2[i][df2['overall_survival']==1], color='darkgreen', label = 'survived')
    sns.distplot(df2[i][df2['overall_survival']==0], color='darkred', label = 'died')
    plt.legend(loc='best')
fig.suptitle('Clinical Data Analysis')
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

From the plots above we know that :

Age at diagonosis
- The patients who survived have bimodal distribution
- The patients who died have left skewed distribution

Lymph nodes examined positive : both the patients who survived and died have right skewed distribution

Mutation count : both the patients who survived and died have right skewed distribution

Nottingham prognostic index : both the patients who survived and died have multimodal distribution 

Overall survival months
- The patients who survived have bimodal distribution
- The patients who died have right skewed distribution 

Tumor size : both the patients who survived and died have right skewed distribution

In [ ]:
# Visualizations Clinical Columns in the Dataframe
died = df2[df2['overall_survival']==0]
survived = df2[df2['overall_survival']==1]

alive_from_cancer = df2[df2['death_from_cancer']=='Living']
died_from_cancer = df2[df2['death_from_cancer']=='Died of Disease']
died_not_cancer = df2[df2['death_from_cancer']=='Died of Other Causes']

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,4), sharey=True)

sns.boxplot(x='overall_survival_months', y='overall_survival', orient='h', data=df2, ax=ax[0], palette = two_colors, saturation=0.90)
sns.boxplot(x='age_at_diagnosis', y='overall_survival', orient='h', data=df2, ax=ax[1], palette = two_colors, saturation=0.90)

fig.suptitle('The Distribution of Survival time and Age at diagnosis', fontsize = 16)

ax[0].set_xlabel('Survival time (Month)')
ax[0].set_ylabel('Survival')
ax[1].set_xlabel('Age at diagnosis (Year)')
ax[1].set_ylabel('')

plt.show()

Based on the boxplots above, we can see that :
- The difference between the two distributions in age_at_diagnosis column, as patients who were older when diagnosed with breast cancer were more likely to not survive. 
- The duration from the time of the intervention to death or to current time is longer in the patients who survive. That means that patients are either dying early from breast cancer or surviving.

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15,4), sharey=True)

sns.boxplot(x='overall_survival_months', y='death_from_cancer', orient='h', data=df2, ax=ax[0], palette = three_colors, saturation=0.90)
sns.boxplot(x='age_at_diagnosis', y='death_from_cancer', orient='h', data=df2, ax=ax[1], palette = three_colors, saturation=0.90)

fig.suptitle('The Distribution of Survival time and Age at diagnosis with Target Attribute', fontsize = 16)

ax[0].set_xlabel('Survival time (month)')
ax[0].set_ylabel('survival')
ax[1].set_xlabel('Age at diagnosis (year)')
ax[1].set_ylabel('')

plt.show()

Based on the boxplots above, we can see that :
- Patients who survived from breast are more likely to survive longer compared to patients who died because of breast cancer and person who died because of other disease. Patients with breast cancer are more likely to die faster than a person who died because of other disease.
- Patients whose age at diagnosis > 60 are more likely to die because of other disease. Patients who were younger tend to die because of breast cancer or survive from breast cancer

In [ ]:
fig, ax = plt.subplots( figsize=(12, 6))
ax = sns.boxplot(x ='tumor_stage', y ='tumor_size',  data = df2, orient='v', hue='overall_survival', palette=two_colors)
ax.set_ylabel('Tumor size')
ax.set_xlabel('Tumor stage')
plt.title('Tumor size distribution per tumor stage', fontsize=16)
plt.show()

Stage 0.0 
- Patients who died have same tumor size, around 60-70
- Patients who survived have tumor size between 15-35

Stage 1.0 
Patients who died and survived has relatively same tumor size distribution, except 1 died patients whose tumor size > 150

Stage 2.0
Patients who died and survived has a little bit difference in tumor size distribution and has almost the same median

Stage 3.0
- Patients who died have variety of tumor size, from around 10 to more than 175
- Patients who survived have tumor size between around 20 and 100

Stage 4.0
- Patients who died have tumor size between 20-70
- Patients who survived have same tumor size, around 60-70

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12,4), sharey=True)

fig.suptitle('The Distribution of Survival and Recurrence with Target Attribute', fontsize = 18)

ax[0].hist(died['overall_survival_months'], alpha=0.9, color=sns.color_palette(color)[0], label='Died')
ax[0].hist(survived['overall_survival_months'], alpha=0.9, color=sns.color_palette(color)[5], label='Survived')
ax[0].legend()

ax[1].hist(alive_from_cancer['overall_survival_months'], alpha=0.9, color=sns.color_palette(color)[5], label='Survived')
ax[1].hist(died_from_cancer['overall_survival_months'], alpha=0.9, color=sns.color_palette(color)[0], label='Died from cancer')
ax[1].hist(died_not_cancer['overall_survival_months'], alpha=0.9, color=sns.color_palette(color)[1], label='Died not from cancer')
ax[1].legend()

ax[0].set_xlabel('Overall survival time (month)')
ax[0].set_ylabel('Number of patients')
ax[1].set_xlabel('Overall survival time (month)')
ax[1].set_ylabel('')

plt.show()

From the plots above we know that :
- The higher survival time, the higher number of patients who survived
- The higher survival time, the less number of patients who died not from cancer

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(14,2), sharey=True)

sns.boxplot(x='tumor_size', y='overall_survival', orient='h', data=df2, ax=ax[0], palette = two_colors, saturation=0.90)
sns.boxplot(x='lymph_nodes_examined_positive', y='overall_survival', orient='h', data=df2, ax=ax[1], palette = two_colors, saturation=0.90)

fig.suptitle('The Distribution of Continuous Clinical Attributes', fontsize = 14)
plt.yticks([-0.5, 0, 1, 1.5], ['','Died', 'Survived',''])

ax[0].set_xlabel('Tumour diameter (mm)')
ax[0].set_ylabel('')

ax[1].set_xlabel('Number of positive lymph nodes')
ax[1].set_ylabel('')

plt.show()

From the plots above, we know that :
- The tumour diameter for patients who died and survived has same minimum and almost the same maximum. THere are many outliers in both distributions
- The minimum number of positive lymph nodes for patient who survived same as the median.
- The minimum number of positive lymph nodes for patient who died same as the Q1
- The distribution of number of positive lymph nodes for died patients is wider than survived patiens

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(15,3))
fig.suptitle('The Distribution of Continuous Clinical Attributes', fontsize = 18)

ax[0].hist(survived['age_at_diagnosis'], alpha=0.9, color=sns.color_palette(color)[5], label='Survived')
ax[0].hist(died['age_at_diagnosis'], alpha=0.9, color=sns.color_palette(color)[0], label='Died')
ax[0].legend()

ax[1].hist(survived['tumor_size'], alpha=0.9, color=sns.color_palette(color)[5], label='Survived')
ax[1].hist(died['tumor_size'], alpha=0.9, color=sns.color_palette(color)[0], label='Died')
ax[1].legend()

ax[2].hist(survived['lymph_nodes_examined_positive'], alpha=0.9, color=sns.color_palette(color)[5], label='Survived')
ax[2].hist(died['lymph_nodes_examined_positive'], alpha=0.9, color=sns.color_palette(color)[0], label='Died')
ax[2].legend()

ax[0].set_xlabel('Age at diagnosis (year)')
ax[0].set_ylabel('Number of patients')
ax[1].set_xlabel('Tumour diameter (mm)')
ax[1].set_ylabel('')
ax[2].set_xlabel('Number of positive nodes')
ax[2].set_ylabel('')

plt.show()

- The median of tumor size and the number of positive lymph nodes is lower in the survived class than the died class.
- The number of patients for age at diagnosis of died class is hihger than survived class

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(17,3))
fig.suptitle('The Distribution of Continuous Clinical Attributes', fontsize = 18)

ax[0].hist(alive_from_cancer['age_at_diagnosis'], alpha=0.8, color=sns.color_palette(color)[5], label='Survived')
ax[0].hist(died_from_cancer['age_at_diagnosis'], alpha=0.8, color=sns.color_palette(color)[0], label='Died from cancer')
ax[0].hist(died_not_cancer['age_at_diagnosis'], alpha=0.8, color=sns.color_palette(color)[1], label='Died not from cancer')
ax[0].legend()

ax[1].hist(alive_from_cancer['tumor_size'], alpha=0.8, color=sns.color_palette(color)[5], label='Survived')
ax[1].hist(died_from_cancer['tumor_size'], alpha=0.8, color=sns.color_palette(color)[0], label='Died from cancer')
ax[1].hist(died_not_cancer['tumor_size'], alpha=0.8, color=sns.color_palette(color)[1], label='Died not from cancer')
ax[1].legend()

ax[2].hist(survived['lymph_nodes_examined_positive'], alpha=0.8, color=sns.color_palette(color)[5], label='Survived')
ax[2].hist(died_from_cancer['lymph_nodes_examined_positive'], alpha=0.8, color=sns.color_palette(color)[0], label='Died from cancer')
ax[2].hist(died_not_cancer['lymph_nodes_examined_positive'], alpha=0.8, color=sns.color_palette(color)[1], label='Died not from cancer')
ax[2].legend()

ax[0].set_xlabel('Age (year)')
ax[0].set_ylabel('Number of patients')
ax[1].set_xlabel('Tumour diameter (mm)')
ax[1].set_ylabel('')
ax[2].set_xlabel('Number of positive nodes')
ax[2].set_ylabel('')

plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(15,3))
fig.suptitle('The Distribution of treatment and survival', fontsize = 18)

sns.countplot(died['chemotherapy'], color=sns.color_palette(color)[0], label='Died', ax=ax[0], saturation=0.90)
sns.countplot(x=survived['chemotherapy'] , color=sns.color_palette(color)[5], label='Survived', ax=ax[0], saturation=0.90)

#ax[0].legend()
ax[0].set(xticklabels=['No','Yes'])

sns.countplot(died['hormone_therapy'], color=sns.color_palette(color)[0], label='Died', ax=ax[1], saturation=0.90)
sns.countplot(x=survived['hormone_therapy'], color=sns.color_palette(color)[5], label='Survived', ax=ax[1], saturation=0.90)

ax[1].legend()
ax[1].set(xticklabels=['No','Yes'])

sns.countplot(died['radio_therapy'], color=sns.color_palette(color)[0], label='Died', ax=ax[2], saturation=0.90)
sns.countplot(x=survived['radio_therapy'], color=sns.color_palette(color)[5], label='Survived', ax=ax[2], saturation=0.90)

#ax[2].legend()
ax[2].set(xticklabels=['No','Yes'])

ax[0].set_xlabel('Chemotherapy')
ax[0].set_ylabel('Number of patients')
ax[1].set_xlabel('Hormonal therapy')
ax[1].set_ylabel('')
ax[2].set_xlabel('Radio therapy')
ax[2].set_ylabel('')

plt.show()

Chemotherapy treatment 
- The difference between died class and survived class is large for patient with chemotherapy compared to patients without chemotherapy
- Patients in died class without chemotherapy is very high, more than 800. Meanwhile, patients in died class with chemotherapy is not so high, a little over 200

Hormonal therapy 
- The difference between died class and survived class is quite large for patient with hormonal therapy compared to patients without hormonal therapy
- Patients in died class with chemotherapy is quite high, a little bit under 700. Patients in died class with chemotherapy is not so high, a little over 400

Radio therapy 
- The difference between died class and survived class is quite large for patient with radio therapy compared to patients without radio therapy
- Number of patients in died class both with radio therapy and without radio therapy is the highest compared to other therapy method

Visualise the treatments and proportion death for other groups using venn diagram

In [ ]:
# create subsets for different combinations of treatments
chemo = df2[(df2["chemotherapy"]==True) & (df2["radio_therapy"]==False) & (df2["hormone_therapy"]==False)]
radio = df2[(df2["chemotherapy"]==False) & (df2["radio_therapy"]==True) & (df2["hormone_therapy"]==False)]
hormonal = df2[(df2["chemotherapy"]==False) & (df2["radio_therapy"]==False) & (df2["hormone_therapy"]==True)]
chemo_radio = df2[(df2["chemotherapy"]==True) & (df2["radio_therapy"]==True) & (df2["hormone_therapy"]==False)]
radio_hormonal = df2[(df2["chemotherapy"]==False) & (df2["radio_therapy"]==True) & (df2["hormone_therapy"]==True)]
hormonal_chemo = df2[(df2["chemotherapy"]==True) & (df2["radio_therapy"]==False) & (df2["hormone_therapy"]==True)]
all_3 = df2[(df2["chemotherapy"]==True) & (df2["radio_therapy"]==True) & (df2["hormone_therapy"]==True)]

#calculate number of people for each combination and proportion death
df_subsets = [chemo, radio, hormonal, chemo_radio, radio_hormonal, hormonal_chemo, all_3]
sizes=[]
proportiondeath=[]
for dataframe in df_subsets:
    sizes.append(np.shape(dataframe)[0])
    proportiondeath.append(np.mean(dataframe["overall_survival"]))

#set size of circles relative to size of each subset (where possible)
#set gradient of blue according to proportion of death in subset calculated above
fig, ax = plt.subplots(figsize=(8,6))
v = venn3(subsets=sizes, set_labels=("Chemo", "Radio ", "Hormonal"), ax=ax, alpha=0.6, set_colors= sns.color_palette(color))

for text in v.set_labels:
    text.set_fontsize(14)
    
ax.set_title("Patients by treatment group", size=20)
plt.show()

- Most of patients get combination of chemotherapy and hormonal therapy treatment, followed by radio therapy and chemotherapy
- Radio therapy is the most often given to the patients in terms of sole treatment
- There are no patients who were given the combination of radio therapy and hormonal therapy

In [ ]:
fig, ax = plt.subplots( figsize=(10,5))
fig.suptitle('The Distribution histopathological class and survival', fontsize = 18)

sns.countplot(x='neoplasm_histologic_grade', hue='overall_survival' ,data = df2, palette=two_colors , ax=ax, saturation=0.90)
ax.legend([ 'Died', 'Survived'])

ax.set_xlabel('histopathological class')
ax.set_ylabel('Number of patients')

plt.show()

- The patient in died class is always higher than in survived class except for class 1.0
- The difference of number of patients both died and survived class for class 1.0 is high compared to other class

## Correlation between attributes

In [ ]:
fig, axs = plt.subplots(figsize = (13, 10)) 
categorical_columns = df2.select_dtypes(include=['object']).columns.tolist()
unwanted_columns = ['patient_id','death_from_cancer' ]
categorical_columns = [ele for ele in categorical_columns if ele not in unwanted_columns] 
no_id_df = pd.get_dummies(df2.drop('patient_id',axis=1 ), columns= categorical_columns)
#no_id_clinical_df= clinical_df.drop('ID',axis=1 )
mask = np.triu(np.ones_like(no_id_df.corr(), dtype = np.bool))
sns.heatmap(no_id_df.corr(), ax = axs, mask = mask, cmap = sns.diverging_palette(180, 10, as_cmap = True))
plt.title('Correlation between the Clinical Attributese')

plt.show()

From correlation plot above we could see that there are some attributes which have positive and negative correlation with other attributes. Beside that some attributes have strong correlation with other attributes while some are not.

In [ ]:
# Correlation attributes and overall survival
survival_corr = no_id_df.corr()['overall_survival'].sort_values(ascending = False)
survival_corr_df = pd.DataFrame({'Correlation': survival_corr})
survival_corr_df.head(10)

All variables which have positive correlation with overall survival is weakly correlated (max correlation < 0.4)

In [ ]:
survival_corr_df.tail()

All variables which have negative correlation with overall survival is weakly correlated (min correlation > -3.1)

Statistical summaries of numeric attributes in the Dataframe

In [ ]:
# Statistical summaries of numeric attributes
num_attributes_columns= ['age_at_diagnosis', 'lymph_nodes_examined_positive','mutation_count',
                       'nottingham_prognostic_index', 'overall_survival_months', 'tumor_size' ]
cat_attributes_columns = ['chemotherapy', 'cohort', 'neoplasm_histologic_grade','hormone_therapy', 
                        'overall_survival', 'radio_therapy', 'tumor_stage' ]
# Statistical summary for numerical clinical attributes 
df2[num_attributes_columns].describe().T 

Statistical summaries of categorical attributes in the Dataframe

In [ ]:
# statistical summaries of categorical attributes
cat_attributes_columns.extend(df2.select_dtypes(include=['object']).columns.tolist())
df2[cat_attributes_columns].astype('category').describe().T

In [ ]:
# statistics for the no treatment group and comparison with the baseline
no_treatment = df2[(df2['chemotherapy']==0) & (df2['hormone_therapy']==0) & (df2['radio_therapy']==0)]
print("Number of patients who had no treatment: " , no_treatment.shape[0])
print("Proportion of survival with no treatment: " , ("%.3f" %np.mean(no_treatment["overall_survival"])))
print("Baseline proportion of survival in all groups: ", ("%.3f" %np.mean(df["overall_survival"])))

The proportion of survival with no treatment is quite close to the baseline proportion of survival in all group

## Characteristics of the patients in the data

In [ ]:
#What the average patient looks like
print("Mean of age : " + "%.3f" %np.mean(df2['age_at_diagnosis']))
print("Most occurring tumour stage :", stats.mode(df2['tumor_stage'])[0][0].astype(int))
print("Most occurring histopathological type :", stats.mode(df2['neoplasm_histologic_grade'])[0][0].astype(int))
print("Mean of tumour diameter : " + "%.3f" %np.mean(df2['tumor_size']))
print("Probability of survival : "+ "%.3f" %(df2["overall_survival"].value_counts()/df2["overall_survival"].count()).iloc[1])

In [ ]:
# Finding number of outliers in each column
Q1 = df2.quantile(0.25)
Q3 = df2.quantile(0.75)
IQR = Q3 - Q1
((df2 < (Q1 - 1.5 * IQR)) | (df2 > (Q3 + 1.5 * IQR))).sum().sort_values(ascending = False)

Some attributes have many outliers

In [ ]:
# Relationship between genetic attributes and outcomes
# dropping mutations
genetic_features_to_drop = df.columns[520:]
df3 = df.drop(genetic_features_to_drop, axis=1)
# droping clinical data
genetic_features_to_drop = df3.columns[4:35]
df3 = df3.drop(genetic_features_to_drop, axis=1)
df3 = df3.drop(['age_at_diagnosis','type_of_breast_surgery', 'cancer_type'], axis=1)
df3 = df3.iloc [:,:-174]
df3['overall_survival']= df['overall_survival']

df3.head()

In [ ]:
#Finding maximum and std in each column, std is always 1 because the datapoints are z-scores
max_values = df3.iloc[:, 1:].max()
std = df3.iloc[:, 1:].std(axis = 0, skipna = True)
max_data = pd.concat([max_values, std], axis = 1, keys = ['max_values', 'std']).sort_values(by='max_values', ascending = False).head()
max_data

In [ ]:
# Visualizing the mRNA values in a heatmap.
fig, axs = plt.subplots(figsize = (17, 10)) 
sns.heatmap(df3.drop(['patient_id','overall_survival'], axis=1), ax = axs, cmap = sns.diverging_palette(180, 10, as_cmap = True))
plt.title('Gene Expression Heatmap')

plt.show()

In [ ]:
params = {'axes.titlesize':'10',
          'xtick.labelsize':'9',
          'ytick.labelsize':'9'}
matplotlib.rcParams.update(params)
#plt.subplots_adjust(hspace=0.5) 
df3.drop(['patient_id','overall_survival'], axis=1).iloc[:,:9].hist(figsize=(15,8), color=color_hist)
plt.show()

In [ ]:
fig = plt.figure(figsize = (20, 25))
j = 0

gene_list = ['rab25', 'eif5a2', 'pik3ca', 'kit', 'fgf1', 'myc', 'egfr', 'notch3', 'kras', 'akt1', 'erbb2', 'pik3r1', 'ccne1', 'akt2', 'aurka']
for i in df3.drop(['patient_id'], axis=1).loc[:,gene_list].columns:
    plt.subplot(6, 4, j+1)
    j += 1
    sns.distplot(df3[i][df3['overall_survival']==0], color='g', label = 'survived')
    sns.distplot(df3[i][df3['overall_survival']==1], color='r', label = 'died')
    plt.legend(loc='best')
fig.suptitle('Genetical attributes distribution')
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

All of the attributes have relatively the same distribution shape for two classes (survival and dead)

In [ ]:
print('Maximum value possible in genetic data:', df3.drop(['patient_id','overall_survival'], axis = 1).max().max())
print('Minimum value possible in genetic data:', df3.drop(['patient_id','overall_survival'], axis = 1).min().min())

Number of outliers in the top 10 genetic features


In [ ]:
#Finding number of outliers in each column
Q1 = df3.quantile(0.25)
Q3 = df3.quantile(0.75)
IQR = Q3 - Q1
((df3 < (Q1 - 1.5 * IQR)) | (df3 > (Q3 + 1.5 * IQR))).sum().sort_values(ascending = False).head(10)

Visualize Correlation of between the genetic Attributes and outcome


In [ ]:
#how varied are genes and how well do they correlate with eventdeath?
fig, ax = plt.subplots(figsize=(10,4))

#plot histogram of variation using standard deviation as a measure
corrs=[]
for col in df3.drop(['patient_id'], axis = 1).columns:
    corr = df3[[col,'overall_survival']].corr()['overall_survival'][col]
    corrs.append(corr)

corrs.pop(-1)
ax.hist(corrs,  bins=25, color = color_hist)
ax.set_xlabel("Correlation")
ax.set_ylabel("Number of genes")
ax.set_title("Histogram of Correlation of genes with the survival", size=16)
plt.show()

In [ ]:
#Max, min and mean of correlation
print("Maximum Correlation: " + "%.3f" %max(corrs))
print("Minimum Correlation: " + "%.3f" %min(corrs))
print("Mean Correlation: " + "%.3f" %np.mean(corrs))

Most of the attributes have very weak correlation with target attributes

In [ ]:
# droping clinical and genetic data
mutation_features_to_drop = df.columns[4:520]
df4 = df.drop(mutation_features_to_drop, axis=1)
df4 = df4.drop(['age_at_diagnosis','type_of_breast_surgery', 'cancer_type'], axis=1)

# if there is a mutation=1, no-mutation=0
for column in df4.columns[1:]:
    df4[column]=pd.to_numeric(df4[column], errors='coerce').fillna(1).astype(int)

df4.insert(loc=1 , column='overall_survival', value=df['overall_survival'])

df4.head()

Some genes had much more mutations than other genes. For example: PIK3CA (coding mutations in 40.1% of the samples) and TP53 (35.4%) dominated the mutation landscape. Only five other genes harboured coding mutations in at least 10% of the samples: MUC16 (16.8%); AHNAK2 (16.2%); SYNE1 (12.0%); KMT2C (also known as MLL3; 11.4%) and GATA3 (11.1%).

In [ ]:
#plot histogram of variation using standard deviation as a measure
fig, ax = plt.subplots(figsize=(10,4))
corrs=[]
for col in df4.drop(['patient_id'], axis = 1).columns:
    corr = df4[[col,'overall_survival']].corr()['overall_survival'][col]
    corrs.append(corr)
    
corrs.pop(0)
ax.hist(corrs,  bins=25, color = color_hist)
ax.set_xlabel("Correlation")
ax.set_ylabel("Number of genes")
ax.set_title("Histogram of Correlation of genes with the survival", size=16)


plt.show()

Most of correlation is between -0.02 and 0.02

# Preprocessing and Modelling

In [ ]:
BOLD = '\033[1m'
END = '\033[0m'
# using a stratfied k fold because we need the distribution of the to classes in all of the folds to be the same.
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
print('Baseline accuracy:' )
print(df["overall_survival"].value_counts()/df["overall_survival"].count())

## Classification with only clinical attributes


In [ ]:
categorical_columns = df2.select_dtypes(include=['object']).columns.tolist()
unwanted_columns = ['patient_id','death_from_cancer' ]
categorical_columns = [ele for ele in categorical_columns if ele not in unwanted_columns] 
# Getting dummies for all categorical columns
dummies_df = pd.get_dummies(df2.drop('patient_id',axis=1 ), columns= categorical_columns, dummy_na=True)
dummies_df.dropna(inplace = True)

In [ ]:
# data splitting
X = dummies_df.drop(['death_from_cancer', 'overall_survival'], axis=1)
y = dummies_df['overall_survival']
# using stratify for y because we need the distribution of the two classes to be equal in train and test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify = y)

Additional functions 

In [ ]:
def model_metrics(model, kfold, X_train, X_test, y_train, y_test):
    
    model.fit(X_train, y_train)

    #metrics
    results = cross_val_score(model, X_train, y_train, cv = kfold)
    print("CV scores: ", results); print("CV Standard Deviation: ", results.std()); print();
    print('CV Mean score: ', results.mean()); 
    print('Train score:   ', model.score(X_train, y_train))
    print('Test score:    ', model.score(X_test, y_test))
    
    pred = model.predict(X_test)
    print("\n")
    print()
    print('Confusion Matrix: ')
    print(confusion_matrix(y_test, pred))
    print('Classification Report:  ')
    print(classification_report(y_test, pred))
    train_score =  model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    test_pred = model.predict(X_test)
    return test_pred, test_score, results.mean()

def basic_classifiers (X_train, X_test, y_train, y_test, kfold):
    BOLD = '\033[1m'
    END = '\033[0m'
    
    # Scaling 
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    ######################################################################################################  K Neighbors Classifier model
    
    params = {
    "n_neighbors" : [5,15,25,30,35,40, 100],
    "weights" : ["uniform" , "distance"]
    }
    print(); print(BOLD + 'K Neighbors Classifier Model:' + END)
    knn= GridSearchCV(KNeighborsClassifier(), params, n_jobs=-1, cv=4)
    knn_pred, knn_test, knn_train = model_metrics(knn, kfold, X_train, X_test, y_train, y_test)
    
    ###################################################################################################### Logistic Regression
    params = {
    "penalty": ["l1", "l2"],
    "C": np.logspace(-2,4,100)
    }
    print(); print(BOLD + 'Logistic Regression Model:' + END)
    logistic_regression = GridSearchCV(LogisticRegression(random_state=42), params, n_jobs=-1, cv=4)
    lg_pred, lg_test, lg_train = model_metrics(logistic_regression, kfold, X_train, X_test, y_train, y_test)
    
    ###################################################################################################### Decision Tree
    
    print(); print(BOLD + 'Decision Tree Classifier Model:' + END)
    decision_tree = DecisionTreeClassifier(random_state=42)
    dt_pred, dt_test, dt_train = model_metrics(decision_tree, kfold, X_train, X_test, y_train, y_test)
    
    ###################################################################################################### Random Forest Classifier
    
    print(); print(BOLD + 'Random Forest Classifier Model:' + END)
    random_forest = RandomForestClassifier(random_state=42)
    rf_pred, rf_test, rf_train = model_metrics(random_forest, kfold, X_train, X_test, y_train, y_test)
    
    ###################################################################################################### Extra Trees Classifier
   
    print(); print(BOLD + 'Extra Trees Classifier Model:' + END)
    extra_trees = ExtraTreesClassifier(random_state=42)
    et_pred, et_test, et_train = model_metrics(extra_trees, kfold, X_train, X_test, y_train, y_test)
    
    ###################################################################################################### AdaBoost Classifier
    
    print(); print(BOLD + 'AdaBoost Classifier Model:' + END)
    ada_boost = AdaBoostClassifier(random_state=42)
    ab_pred, ab_test, ab_train = model_metrics(ada_boost, kfold, X_train, X_test, y_train, y_test)
    
    ###################################################################################################### SVC Classifier
    
    print(); print(BOLD + 'SVC Classifier Model:' + END)
    svc = SVC(random_state=42)
    svc_pred, svc_test, svc_train = model_metrics(svc, kfold, X_train, X_test, y_train, y_test)

    fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(15,6))

    
    #bar chart of accuracy scores
    inds = range(1,8)
    labels = ["KNN", "Logistic Regression", "Decision Tree", "Random Forest",'Extra Trees', 'AdaBoost', 'SVC' ]
    scores_all = [knn_train, lg_train, dt_train, rf_train, et_train, ab_train, svc_train]
    scores_predictive = [knn_test, lg_test, dt_test, rf_test, et_test, ab_test, svc_test]
    
    ax1.bar(inds, scores_all, color=sns.color_palette(color)[5], alpha=0.3, hatch="x", edgecolor="none",label="CrossValidation Set")
    ax1.bar(inds, scores_predictive, color=sns.color_palette(color)[0], label="Testing set")
    ax1.set_ylim(0.4, 1)
    ax1.set_ylabel("Accuracy score")
    ax1.axhline(0.5793, color="black", linestyle="--")
    ax1.set_title("Accuracy scores for basic models", fontsize=17)
    ax1.set_xticks(range(1,8))
    ax1.set_xticklabels(labels, size=12, rotation=40, ha="right")
    ax1.legend()

    labels = ["KNN", "Logistic Regression", "Decision Tree", "Random Forest",'Extra Trees', 'AdaBoost', 'SVC' ]
    for label, pred in zip(labels, [knn_pred, lg_pred, dt_pred, rf_pred, et_pred, ab_pred, svc_pred]):
        fpr, tpr, threshold = roc_curve(y_test.values, pred)
        roc_auc = auc(fpr, tpr)
        ax2.plot(fpr, tpr, label=label+' (area = %0.2f)' % roc_auc, linewidth=2)
    ax2.plot([0, 1], [0, 1], 'k--', linewidth=2)
    ax2.set_xlim([-0.05, 1.0])
    ax2.set_ylim([-0.05, 1.05])
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.legend(loc="lower right", prop={'size': 12})
    ax2.set_title("Roc curve for for basic models", fontsize=17)

    plt.show()
    
    
# a function that takes a dataframe and plots histograms for all columns 
def subplot_histograms(dataframe, list_of_columns, list_of_titles, list_of_xlabels, big_title_name):
    
    nrows = int(np.ceil(len(list_of_columns)/3)) # Makes sure you have enough rows
    fig, ax = plt.subplots(ncols=3,nrows=nrows, figsize=(15, 10)) # You'll want to specify your figsize
    fig.suptitle(big_title_name, fontsize=15)
    ax = ax.ravel() # Ravel turns a matrix into a vector, which is easier to iterate
    for i, column in enumerate(list_of_columns): # Gives us an index value to get into all our lists
        ax[i].hist(dataframe[column].dropna(), color= color_hist ) # feel free to add more settings
        #ax[i].set_xlabel(list_of_xlabels[i])
        ax[i].set_ylabel('Frequency')
        ax[i].set_title(list_of_titles[i]) # Set titles, labels, etc here for each subplot    
    plt.show()
    
    
# a function that takes a dataframe and plots barplot for all columns 
def subplot_bargraph(dataframe, list_of_columns, list_of_titles, list_of_xlabels, big_title_name):
    
    nrows = int(np.ceil(len(list_of_columns)/3)) # Makes sure you have enough rows
    fig, ax = plt.subplots(ncols=3,nrows=nrows, figsize=(15, 10)) # You'll want to specify your figsize
    fig.suptitle(big_title_name, fontsize=20)
    ax = ax.ravel() # Ravel turns a matrix into a vector, which is easier to iterate
    for i, column in enumerate(list_of_columns): # Gives us an index value to get into all our lists
        sns.countplot(dataframe[column].dropna(), color= color_hist, ax=ax[i], hue=dataframe['eventdeath']) # feel free to add more settings
        #ax[i].set_xlabel(list_of_xlabels[i])
        ax[i].set_xlabel('')
        ax[i].set_ylabel('Frequency')
        ax[i].set_title(list_of_titles[i]) # Set titles, labels, etc here for each subplot    
    plt.show()   

In [ ]:
basic_classifiers( X_train, X_test, y_train, y_test, kfold)

Logistic regression model preformed the best with accuracy of 0.777 and AUC of 0.777, KNN having the lowest accuracy of 0.64, and AUC of 0.62

In [ ]:
def RandomForest_GridSearch(X_train, X_test, y_train, y_test, kfold):
    BOLD = '\033[1m'
    END = '\033[0m'
    print(); print(BOLD + 'Grid Search with Random Forest Classifier Model:' + END)
    #kfold=5
    rf_params = {
        #'n_estimators': [10, 50, 100, 150, 200, 250],
        'max_features':[2, 3, 5, 7, 8],
        #'max_depth': [1, 2, 3, 4, 5, 8],
        #'criterion':['gini', 'entropy'],
    }

    random_forest = RandomForestClassifier(n_estimators=100)
    gs = GridSearchCV(random_forest, param_grid=rf_params, cv=5, verbose = 1)
    gs_pred, gs_test, gs_train = model_metrics(gs, kfold, X_train, X_test, y_train, y_test)
    
    return gs.best_estimator_, gs_pred, gs_test, gs_train


def ExtraTrees_GridSearch(X_train, X_test, y_train, y_test, kfold):
    BOLD = '\033[1m'
    END = '\033[0m'
    print(); print(BOLD + 'Grid Search with Extra Trees Model:' + END)
    # Scaling 
      
    rf_params = {
        #'n_estimators': [10, 100, 400, 800, 1100, 1850],
        #'max_features':['auto'],
        'max_depth': [1, 2, 3, 4, 5, 8],
        #'criterion':['gini'],
    }

    extra_trees = ExtraTreesClassifier(n_estimators=100)    
    gs = GridSearchCV(extra_trees, param_grid=rf_params, cv=5, verbose = 1)
    gs_pred, gs_test, gs_train = model_metrics(gs, kfold, X_train, X_test, y_train, y_test)
    
    return gs.best_estimator_, gs_pred, gs_test, gs_train

def RF_ET_GridSearch (X_train, X_test, y_train, y_test, kfold):
    rf_gs_best_estimator, rf_pred, rf_test, rf_train = RandomForest_GridSearch(X_train, X_test, y_train, y_test, kfold)
    et_gs_best_estimator, et_pred, et_test, et_train = ExtraTrees_GridSearch(X_train, X_test, y_train, y_test, kfold)
    
    fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(13,6))

    fig.suptitle("Random Forest and Extra Trees with Grid Search", fontsize=16)
    #bar chart of accuracy scores
    inds = range(1,3)
    labels = ["Random Forest", "Extra Trees" ]
    scores_all = [rf_train, et_train]
    scores_predictive = [rf_test, et_test]
    
    ax1.bar(inds, scores_all, color=sns.color_palette(color)[5], alpha=0.3, hatch="x", edgecolor="none",label="CrossValidation Set") #
    ax1.bar(inds, scores_predictive, color=sns.color_palette(color)[0], label="Testing set")
    ax1.set_ylim(0.4, 1)
    ax1.set_ylabel("Accuracy score")
    ax1.axhline(0.5793, color="black", linestyle="--")
    ax1.set_title("Accuracy scores", fontsize=17)
    ax1.set_xticks(range(1,3))
    ax1.set_xticklabels(labels, size=14)
    ax1.legend()

    labels = ["Random Forest", "Extra Trees" ]
    for label, pred in zip(labels, [rf_pred, et_pred]):
        fpr, tpr, threshold = roc_curve(y_test.values, pred)
        roc_auc = auc(fpr, tpr)
        ax2.plot(fpr, tpr, label=label+' (area = %0.2f)' % roc_auc, linewidth=2)
    ax2.plot([0, 1], [0, 1], 'k--', linewidth=2)
    ax2.set_xlim([-0.05, 1.0])
    ax2.set_ylim([-0.05, 1.05])
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.legend(loc="lower right", prop={'size': 14})
    ax2.set_title("Roc curve", fontsize=17)

    plt.show()

In [ ]:
RandomForest_GridSearch(X_train, X_test, y_train, y_test, kfold)

Grid Search with Random Forest Classifier Model Accuracy = 0.7384259259259259

Grid Search with Extra Trees Classifier Model Accuracy = 0.7263636363636363

Predicting without the time related column (overall_survival_months)

In [ ]:
# data splitting
X_no_time = dummies_df.drop(['death_from_cancer', 'overall_survival','overall_survival_months' ], axis=1)
y_no_time = dummies_df['overall_survival']

X_train_no_time, X_test_no_time, y_train_no_time, y_test_no_time = train_test_split(X_no_time, y_no_time, test_size=0.33, random_state=42, stratify = y)

In [ ]:
basic_classifiers( X_train_no_time, X_test_no_time, y_train_no_time, y_test_no_time, kfold)

Without survival time duration, the preductor for overall survival performs worse than predictor with survival time duration

In [ ]:
RandomForest_GridSearch(X_train, X_test, y_train, y_test, kfold)

## Classification with only genetic attributes

In [ ]:
# data splitting
X = df3.drop(['patient_id', 'overall_survival'], axis=1)
y = df3['overall_survival']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify = y)

In [ ]:
basic_classifiers( X_train, X_test, y_train, y_test, kfold)

When we train the models with genetic data, we can see that the performance is bad in most of the models

In [ ]:
RandomForest_GridSearch(X_train, X_test, y_train, y_test, kfold)

## Classification with all attributes

In [ ]:
features_to_drop = df.columns[520:]
df = df.drop(features_to_drop, axis=1)
all_categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
unwanted_columns = ['patient_id','death_from_cancer' ]
all_categorical_columns = [ele for ele in all_categorical_columns if ele not in unwanted_columns] 
dummies_df = pd.get_dummies(df.drop('patient_id',axis=1 ), columns= all_categorical_columns, dummy_na=True)
dummies_df.dropna(inplace = True)

In [ ]:
# data splitting
X = dummies_df.drop( ['death_from_cancer','overall_survival'], axis=1)
y = dummies_df['overall_survival']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
basic_classifiers( X_train, X_test, y_train, y_test, kfold)

In [ ]:
RandomForest_GridSearch(X_train, X_test, y_train, y_test, kfold)

## Reference

https://www.kaggle.com/code/raghadalharbi/breast-cancer-survival-prediction-acc-0-779
https://www.kaggle.com/code/meisamomidi/prediction-of-breast-cancer-survival
https://www.kaggle.com/code/samueldegbert/mutation-count-and-morbidity-eda-and-forests/notebook?scriptVersionId=106145302